In [10]:
import numpy as np
import pandas as pd
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform

In [11]:
reform = Reform.from_dict(
    {
        "gov.contrib.repeal_state_dependent_exemptions.in_effect": {
            "2024-01-01.2100-12-31": True
        }
    },
    country_id="us",
)


baseline = Microsimulation(dataset="pooled_3_year_cps_2023")
reformed = Microsimulation(reform=reform, dataset="pooled_3_year_cps_2023")
baseline_income = baseline.calculate("household_net_income", period=2024)
reformed_income = reformed.calculate("household_net_income", period=2024)
difference_income = reformed_income - baseline_income

In [12]:
# Get state codes
state_code = baseline.calculate("state_code", period=2024)

# Get household weights for accurate aggregation
weights = baseline.calculate("household_weight", period=2024)

# Create a DataFrame with the results
results_df = pd.DataFrame(
    {"state_code": state_code, "income_change": difference_income, "weight": weights}
)

# Calculate state-level statistics
state_impacts = (
    results_df.groupby("state_code")
    .agg(
        {
            "income_change": lambda x: np.average(
                x, weights=results_df.loc[x.index, "weight"]
            ),
            "weight": "sum",
        }
    )
    .reset_index()
)

In [13]:
# Calculate total income impact
total_impact_by_state = (
    results_df.groupby("state_code")
    .apply(lambda x: np.sum(x["income_change"] * x["weight"]))
    .reset_index(name="total_income_impact")
)

# Merge total impact into main results
state_impacts = state_impacts.merge(total_impact_by_state, on="state_code")

# Add households affected
state_impacts["households_affected"] = (
    results_df[results_df["income_change"] != 0]
    .groupby("state_code")["weight"]
    .sum()
    .reindex(state_impacts["state_code"])
    .fillna(0)
)

# Calculate percentage of households affected
state_impacts["percent_households_affected"] = (
    state_impacts["households_affected"] / state_impacts["weight"] * 100
)

# Format the results
state_impacts["avg_income_change"] = state_impacts["income_change"].round(2)
state_impacts["total_income_impact"] = state_impacts["total_income_impact"].round(0)
state_impacts["households_affected"] = state_impacts["households_affected"].round(0)
state_impacts["percent_households_affected"] = state_impacts[
    "percent_households_affected"
].round(2)
state_impacts["weight"] = state_impacts["weight"].round(0)

# Sort by total impact
state_impacts_sorted = state_impacts.sort_values("total_income_impact")

# Rename columns for clarity
state_impacts_sorted = state_impacts_sorted.rename(
    columns={
        "weight": "total_households",
        "avg_income_change": "avg_income_change_per_household",
        "total_income_impact": "total_state_income_impact",
    }
)

# Calculate national totals
national_totals = pd.DataFrame(
    {
        "state_code": ["TOTAL"],
        "total_households": [state_impacts_sorted["total_households"].sum()],
        "avg_income_change_per_household": [
            (
                state_impacts_sorted["total_state_income_impact"].sum()
                / state_impacts_sorted["total_households"].sum()
            )
        ],
        "total_state_income_impact": [
            state_impacts_sorted["total_state_income_impact"].sum()
        ],
        "households_affected": [state_impacts_sorted["households_affected"].sum()],
        "percent_households_affected": [
            (
                state_impacts_sorted["households_affected"].sum()
                / state_impacts_sorted["total_households"].sum()
                * 100
            )
        ],
    }
)

# Combine state results with national totals
final_results = pd.concat([state_impacts_sorted, national_totals])

# Display results
print("\nState-by-State Impact Analysis:")
print(
    final_results[
        [
            "state_code",
            "total_households",
            "avg_income_change_per_household",
            "total_state_income_impact",
            "households_affected",
            "percent_households_affected",
        ]
    ].to_string()
)


State-by-State Impact Analysis:
   state_code  total_households  avg_income_change_per_household  total_state_income_impact  households_affected  percent_households_affected
4          CA        14365824.0                      -161.660004              -2.322313e+09                  NaN                          NaN
22         MI         4228719.0                      -113.089996              -4.782457e+08                  NaN                          NaN
10         GA         4348380.0                       -94.570000              -4.112280e+08                  NaN                          NaN
23         MN         2344873.0                      -166.809998              -3.911444e+08                  NaN                          NaN
14         IL         5187204.0                       -61.619999              -3.196205e+08                  NaN                          NaN
34         NY         7896529.0                       -28.510000              -2.251366e+08                  NaN   

In [14]:
final_results.to_csv("state_impacts_detailed.csv", index=False)